In [0]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras import Model
import random
from sklearn.neighbors import KNeighborsClassifier
import matplotlib as plt
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# load files
!unzip "/content/drive/My Drive/Flickr8k_Dataset.zip" -d Flickr8k_Dataset
!unzip "/content/drive/My Drive/Flickr8k_text.zip" -d Flickr8k_text

Archive:  /content/drive/My Drive/Flickr8k_Dataset.zip
replace Flickr8k_Dataset/Flicker8k_Dataset/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/My Drive/Flickr8k_text.zip
replace Flickr8k_text/CrowdFlowerAnnotations.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
with open("Flickr8k_text/Flickr_8k.trainImages.txt","r") as f:
    data = f.read()

In [30]:
# get pretrained model
in_layer = Input(shape=(224, 224, 3))
vgg = VGG16(weights='imagenet', input_tensor=in_layer)
print(vgg.summary())

model_output = vgg.get_layer("fc2").output # get output of penultimate layer (fc7)
model = Model(inputs=vgg.input, outputs=model_output)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [31]:
# extract features from each photo in the directory
def extract_features(directory):
	# load the model
	# extract features from each photo
	count = 0
	num_images = 8091
	features = dict()
	for name in listdir(directory):
	# load an image from file
		filename = directory + '/' + name
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		count += 1
		if count % 100 == 0:
			print("%i/%i" % (count, num_images))
	return features

# extract features from all images
directory = 'Flickr8k_Dataset/Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

100/8091
200/8091
300/8091
400/8091
500/8091
600/8091
700/8091
800/8091
900/8091
1000/8091
1100/8091
1200/8091
1300/8091
1400/8091
1500/8091
1600/8091
1700/8091
1800/8091
1900/8091
2000/8091
2100/8091
2200/8091
2300/8091
2400/8091
2500/8091
2600/8091
2700/8091
2800/8091
2900/8091
3000/8091
3100/8091
3200/8091
3300/8091
3400/8091
3500/8091
3600/8091
3700/8091
3800/8091
3900/8091
4000/8091
4100/8091
4200/8091
4300/8091
4400/8091
4500/8091
4600/8091
4700/8091
4800/8091
4900/8091
5000/8091
5100/8091
5200/8091
5300/8091
5400/8091
5500/8091
5600/8091
5700/8091
5800/8091
5900/8091
6000/8091
6100/8091
6200/8091
6300/8091
6400/8091
6500/8091
6600/8091
6700/8091
6800/8091
6900/8091
7000/8091
7100/8091
7200/8091
7300/8091
7400/8091
7500/8091
7600/8091
7700/8091
7800/8091
7900/8091
8000/8091
Extracted Features: 8091


In [32]:
# loads one description per image
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# store the first description for each image
		if image_id not in mapping:
			mapping[image_id] = image_desc
	return mapping

filename = 'Flickr8k_text/Flickr8k.token.txt'
doc = load_doc(filename)
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [35]:
# this version of descriptions has all 5 descriptions for each doc
import string
# extract descriptions for images
def load_full_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# store the first description for each image
		if image_id not in mapping:
			mapping[image_id] = [image_desc]
		else:
			mapping[image_id].append(image_desc)
	return mapping

# clean description text
def clean_full_descriptions(descriptions):
# prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, v in descriptions.items():
    all_desc = []
    for desc in v:
      # tokenize
      desc = desc.split()
      # convert to lower case
      desc = [word.lower() for word in desc]
      # remove punctuation from each token
      desc = [w.translate(table) for w in desc]
      # remove hanging 's' and 'a'
      desc = [word for word in desc if len(word)>1]
      # store as string
      all_desc.append(' '.join(desc))
    descriptions[key] = all_desc

filename = 'Flickr8k_text/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
full_descriptions = load_full_descriptions(doc)
print('Loaded: %d ' % len(full_descriptions))
# clean descriptions
clean_full_descriptions(full_descriptions)


Loaded: 8092 


In [0]:
# generate splits
X_train, X_test, y_train, y_test, y_test_multiple = [], [], [], [], []

files = ["Flickr8k_text/Flickr_8k.trainImages.txt",
         "Flickr8k_text/Flickr_8k.devImages.txt",
         "Flickr8k_text/Flickr_8k.testImages.txt"]
for idx, f_name in enumerate(files):
  f = open(f_name, 'r')
  for line in f:
    image_id = line.strip()[:-4]
    if idx == 0 or idx == 1:
      X_train.append(features[image_id][0])
      y_train.append(descriptions[image_id])
    else:
      X_test.append(features[image_id][0])
      y_test.append(descriptions[image_id])
      y_test_multiple.append(full_descriptions[image_id])
  f.close()


In [37]:
i = 1
clf = KNeighborsClassifier(n_neighbors=i)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
y_pred = clf.predict(X_test)

In [0]:
from nltk.translate.bleu_score import corpus_bleu


In [42]:
y_test_multiple[0]

['the dogs are in the snow in front of fence',
 'the dogs play on the snow',
 'two brown dogs playfully fight in the snow',
 'two brown dogs wrestle in the snow',
 'two dogs playing in the snow']

In [43]:
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
count = 0
score = 0
for y_true, y_hat in zip(y_test, y_pred):
  references = [[y_true.lower().split()]]
  candidates = [y_hat.lower().split()]
  score += corpus_bleu(references, candidates)
  count += 1
print(score / count)

0.4763962696496128


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
